<a href="https://colab.research.google.com/github/ClaudeJang/MAIN/blob/master/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EC%A7%81%EB%B0%A9_part_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf_8 -*- 
# 파일을 utf-8로 바꾸어 준다. 

# Data processing 
import numpy as np 
import pandas as pd 
import os 
# Gradient Boosting 
import lightgbm as lgbm
import xgboost as xgb 

# os.listdir('./Colab Notebooks/data/')
os.chdir('./Colab Notebooks/data/')


pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1) 
# 더 많은 열을 보기 위해 출력 디스플레이를 넓히는 방법 

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv') 
rates = pd.read_csv('interest_rates.csv', index_col = 0)
subway = pd.read_csv('subway_train.csv')

rates=rates[['mortgage loan', 'year_month']] 

In [0]:
rates.columns = rates.columns.str.replace('year_month', 'transaction_year_month')
rates.columns = rates.columns.str.replace('mortgage loan', 'mortgage_loan')

In [0]:
subway.drop(['latitude','longitude'], axis = 1, inplace = True) 

## Data Preprocessing

### Dropping

In [0]:
drop = ['heat_type','heat_fuel','front_door_structure'] 
# 범주형 변수에 해당하는 난방 타입과 연료, 문 구조등을 사전에 제외한다. + 결측치가 존재하는 feature
train.drop(drop, axis = 1, inplace=True)
test.drop(drop, axis =1, inplace=True)

In [0]:
all_ = train.append(test)
# train 1601458 
# test start from 1601458 
all_=pd.merge(all_,rates)

In [0]:
all_

### Feature Engineering

In [0]:
trans_date = {'1~10' : '월초', '11-20':'월중','21-28':'월말', '21~30':'월말','21~31' : '월말'}
all_['transaction_date'] = [trans_date[i] for i in all_['transaction_date']]

In [0]:
all_head()

In [0]:
# 날짜 데이터를 숫자로 변환 
# 월 단위 숫자로 변환. 
# train['transaction_year_month'] = train['transaction_year_month'].astype(str)

def date_to_int(df):
    df['transaction_year_month'] = df['transaction_year_month'].astype(str)
    Y = [int(a[:4]) for a in df['transaction_year_month']]
    M = [int(a[4:]) for a in df['transaction_year_month']]
    int_date=[]
    for i in range(len(Y)):
        int_date.append((Y[i] - 2006) * 12 +M[i])
    df['transaction_int_date'] = int_date
    
    return df
# int(train.transaction_year_month[0][:4])
# train['transaction_int_date'] = train['transaction_year_month'].map(date_to_int).astype(np.int8)


# 날짜 데이터를 수치형 변수가 아니라 좀 더 활용하는 방안을 찾아야 함.

In [0]:
test = date_to_int(all_) # train / test 의 test가 아니라, 의미없는 'test' 이름으로 지정.
test.info()

### Address

In [0]:
test['address_by_law'] = test['address_by_law'].astype(str)
test['district'] = test['address_by_law'].apply(lambda x: x[:5]) # 구별 코드까지 지정. 
test['dong'] = test['address_by_law'].apply(lambda x: x[5:])

In [0]:
# 구별 코드에 따라 권역으로 나눈다. seoul / pusan 

list4 = []
s_center = ['11110','11140','11170']
s_eastnorth = ['11200','11215','11230','11260','11290','11305','11320','11350'] 
s_westnorth = ['11380','11410','11440'] 
s_westsouth = ['11470','11500','11530','11545','11560','11590','11620']
s_eastsouth = ['11650','11680','11710','11740']
p_center = ['26110','26140','26170','26200','26230','26290','26470','26500']
p_east = ['26350','26410','26260','26710']
p_west=['26320','26440','26530','26380']

for i in test['district']:
    if i in s_center:
        list4.append('도심권')
    elif i in s_eastnorth:
        list4.append('동북권')
    elif i in s_westnorth:
        list4.append('서북권')
    elif i in s_westsouth:
        list4.append('서남권')
    elif i in s_eastsouth:
        list4.append('동남권')
    elif i in p_center:
        list4.append('중부산권')
    elif i in p_east:
        list4.append('동부산권')
    else:
        list4.append('서부산권')

In [0]:
test['district_group'] = list4 
# 권역을 새로운 열로 지정. 
test.info()

In [0]:
test['city'] = test['city'].apply(lambda x :'seoul' if x == 1 else 'busan' )
test['floor'] = test['floor'].apply(lambda x : '저층' if x < 8 else '중층' if 8<=x<18 else '고층' if 18<=x<=30 else '초고층') 
test['APT_area'] = (test['exclusive_use_area'] + test['supply_area']) * 0.3025
test['building_year'] = 2018 - test['year_of_completion'] 
# 람다 함수와 if 문을 사용하여 간편하게 category로 만들었습니다. 
test['above_49'] = test['building_year'].apply(lambda x : 'Yes' if x == 49 else 'No' )

In [0]:
# drop = ['transaction_year_month','latitude','longitude','address_by_law','room_id','district','exclusive_use_area','supply_area']

# test.drop(drop, axis= 1, inplace= True) # 남은 구별 수치형 제거 
# test.info()

test.set_index('key',inplace =True)

In [0]:
# Data preprocess and feature engineering  
def apply_transform(test):
    features = ['apartment_id','city', 'transaction_year_month', 'transaction_date',
       'year_of_completion', 'exclusive_use_area', 'floor', 'latitude',
       'longitude', 'address_by_law', 
       'total_household_count_in_sites', 'apartment_building_count_in_sites',
       'room_id',
       'supply_area', 'total_household_count_of_area_type','household_loan',
       'mean_loan', 'mortgage_loan','subway_trans', 'subway_ordi',
       'sub_trans_yesorno', 'sub_ordi_yesorno',
       'transaction_int_date', 'district','dong', 'district_group', 'APT_area',
       'building_year', 'above_49']
    
    # 주차장 결측치를 평균값으로 대체한다. + 방, 화장실, 최고층 동, 최저층 동 
    test['total_parking_capacity_in_site'] = test['total_parking_capacity_in_site'].fillna(test['total_parking_capacity_in_site'].mean()).astype(np.int16)
    features.append('total_parking_capacity_in_site')
    
    test['room_count'] = test['room_count'].fillna(test['room_count'].mean()).astype(np.int16)
    features.append('room_count')
    test['bathroom_count'] = test['bathroom_count'].fillna(test['bathroom_count'].mean()).astype(np.int16)
    features.append('bathroom_count')
    test['tallest_building_in_sites'] = test['tallest_building_in_sites'].fillna(0.0).astype(np.int16)
    features.append('tallest_building_in_sites')
    test['lowest_building_in_sites'] = test['lowest_building_in_sites'].fillna(0.0).astype(np.int16)
    features.append('lowest_building_in_sites')
    target = test['transaction_real_price']
    test.drop(['transaction_real_price'],axis=1, inplace = True)

## 더미변수로 만들지 않고 뒷 부분에서 category로 변수 타입을 바꿉니다. -> lighGBM에서 과적합을 막고 성능을 높이는 방법. 

#     # one-hot encoding 수행()
#     test1=pd.get_dummies(test['transaction_date'],prefix='date')
#     features.append('date_월초')
#     features.append('date_월중')
#     features.append('date_월말')
#     test2=pd.get_dummies(test['city'],prefix='city')
#     features.append('city_seoul')
#     features.append('city_busan')
#     test3=pd.get_dummies(test['floor'],prefix='floor')
#     features.append('floor_저층')
#     features.append('floor_중층')
#     features.append('floor_고층')
#     features.append('floor_초고층')
#     test4=pd.get_dummies(test['district_group'],prefix='district')
#     features.append('district_도심권')
#     features.append('district_동북권')
#     features.append('district_서북권')
#     features.append('district_서남권')
#     features.append('district_동남권')
#     features.append('district_중부산권')
#     features.append('district_서부산권')
#     features.append('district_동부산권')
#     test=pd.concat([test,test1,test2,test3,test4],axis=1) 
#     test=test.drop(columns=['city','floor','district_group','transaction_date'])
    return test, tuple(features), target
    

In [0]:
test, features, target = apply_transform(test) 
# test는 주요 변수를 포함한 훈련/테스트 데이터 통합 데이터프레임 
# features는 학습에 사용하기 위한 변수를 기록한 튜플 

In [0]:
features2=list(features)
test.info()
## 
features2.remove('city')
features2.remove('transaction_year_month')
features2.remove('transaction_date')
features2.remove('floor')
features2.remove('address_by_law')
features2.remove('district')
features2.remove('district_group')
features2.remove('above_49')

In [0]:
cat_cols = [col for col in test.columns if test[col].dtype in ['O']]
cat_cols_df=test[cat_cols]
test.drop(cat_cols, axis=1, inplace=True)
test.info()
# cat_cols
# 카테고리 형 변수로 만들기 위해 object 변수들을 cat_cols에 저장하고 원래 데이터프레임에서 제거 

In [0]:
for i in enumerate(cat_cols):
  ca=i[1]
  print(ca)
  cat_cols_df[ca] = cat_cols_df[ca].astype('category') 
  
 # cat_cols의 타입을 category로 변경. -> lightGBM에서 학습가능하다. 

In [0]:
for i in enumerate(cat_cols):
  ca=i[1]
  print(ca)
  cat_cols_df[ca] = cat_cols_df[ca].astype('category') 
  
 # cat_cols의 타입을 category로 변경. -> lightGBM에서 학습가능하다. 

In [0]:
robust_scaled_df = test
robust_scaled_df

In [0]:
cat_cols_df # reset_index 필요 
# features2

In [0]:
cat_cols_df # reset_index 필요 
# features2

In [0]:
dataframe=pd.concat([robust_scaled_df,cat_cols_df], axis=1)
dataframe.shape
dataframe.info()

In [0]:
dataframe=pd.concat([robust_scaled_df,cat_cols_df], axis=1)
dataframe.shape
dataframe.info()
new_df.to_csv('train_test.csv', index = False) # 결과 csv 파일 저장. 